In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
from matplotlib.ticker import FuncFormatter
import time
from datetime import datetime
import matplotlib.dates as mdates
import linecache
from iminuit import Minuit 
sys.path.append('..')
from read_data_functions import *
from plot_functions import *
from Laura import *
from calculations import *

In [6]:
parent_path = '../../../' 
path = 'L:/PG-Nanoteknologi/PROJEKTER/2024 Laura og Nan/Lund/LCS_vores/'
names_KU = ['DG-0051', 'DG-0049']
names_Lund = ['LC_20241015', 'LC_20241016', 'LC_20241017']
names_OPC5000 = ['LCS109_2024-10-14', 'LCS109_2024-10-15', 'LCS109_2024-10-16', 'LCS109_2024-10-17']

ParentPath = os.path.abspath(parent_path)
if ParentPath not in sys.path:
    sys.path.insert(0, ParentPath)

LCS_KU = read_LCS_KU_data(path, 'timestamp', names_KU)
LCS_LUND = read_LCS_data_LUND(path, 'Timestamp', names_Lund)
LCS_OPC5000 = read_LCS_data_OPC5000(path, 'timestamp', names_OPC5000, ',')

In [9]:
timestamps_BG_before = [['2024-10-16 10:16:00', '2024-10-16 10:26:00'],
                        ['2024-10-16 14:43:00', '2024-10-16 14:54:00'],
                        ['2024-10-17 09:48:00', '2024-10-17 10:00:00'],
                        ['2024-10-17 13:16:00', '2024-10-17 13:26:00']]

timestamps_increase = [['2024-10-16 10:26:00', '2024-10-16 10:45:00'],
                       ['2024-10-16 14:54:00', '2024-10-16 15:00:00'],
                       ['2024-10-17 10:00:00', '2024-10-17 10:20:00'],
                       ['2024-10-17 14:19:00', '2024-10-17 14:30:00']]

timestamps_exp = [['2024-10-16 10:45:00', '2024-10-16 12:26:00'],
                  ['2024-10-16 15:00:00', '2024-10-16 15:34:00'],
                  ['2024-10-17 10:20:00', '2024-10-17 12:01:00'],
                  ['2024-10-17 14:30:00', '2024-10-17 15:01:00']]

timestamps = [timestamps_BG_before, timestamps_increase, timestamps_exp]

In [7]:
print(LCS_KU.keys())
print(LCS_LUND.keys())
print(LCS_OPC5000.keys())

dict_keys(['DG-0051', 'DG-0049'])
dict_keys(['2024-10-15 DG-0070', '2024-10-15 DG-0071', '2024-10-15 DG-0073', '2024-10-15 DG-0074', '2024-10-15 DG-0076', '2024-10-15 DG-0078', '2024-10-15 DG-0080', '2024-10-15 DG-0081', '2024-10-15 DG-0082', '2024-10-15 DG-0083', '2024-10-15 DG-0085', '2024-10-15 DG-0072', '2024-10-15 DG-0077', '2024-10-16 DG-0070', '2024-10-16 DG-0071', '2024-10-16 DG-0072', '2024-10-16 DG-0073', '2024-10-16 DG-0074', '2024-10-16 DG-0076', '2024-10-16 DG-0077', '2024-10-16 DG-0078', '2024-10-16 DG-0080', '2024-10-16 DG-0081', '2024-10-16 DG-0082', '2024-10-16 DG-0083', '2024-10-16 DG-0085', '2024-10-16 DG-0075', '2024-10-17 DG-0070', '2024-10-17 DG-0071', '2024-10-17 DG-0072', '2024-10-17 DG-0073', '2024-10-17 DG-0074', '2024-10-17 DG-0075', '2024-10-17 DG-0076', '2024-10-17 DG-0077', '2024-10-17 DG-0078', '2024-10-17 DG-0080', '2024-10-17 DG-0081', '2024-10-17 DG-0082', '2024-10-17 DG-0083', '2024-10-17 DG-0085'])
dict_keys(['LCS109_2024-10-14', 'LCS109_2024-10-15',

In [14]:
LCS_conc = ['PM5000S_2_PN0.3', 'PM5000S_2_PN0.5', 'PM5000S_2_PN1', 'PM5000S_2_PN2.5', 'PM5000S_2_PN5', 'PM5000S_2_PN10']
KU_dict_keys =[['DG-0049', 'DG-0049', 'DG-0049', 'DG-0049'], 
               ['DG-0051', 'DG-0051', 'DG-0051', 'DG-0051']]
OPC5000_dict_keys = ['LCS109_2024-10-16', 'LCS109_2024-10-16', 'LCS109_2024-10-17', 'LCS109_2024-10-17']
Lund_conc = ['PN0.5, #/m3', 'PN1, #/cm3', 'PN2.5, #/cm3', 'PN4, #/cm3', 'PN10, #/cm3']
Lund_dict_keys = [['2024-10-16 DG-0072', '2024-10-16 DG-0072', '2024-10-17 DG-0072', '2024-10-17 DG-0072'],
                  ['2024-10-16 DG-0077', '2024-10-16 DG-0077', '2024-10-17 DG-0077', '2024-10-17 DG-0077']]

In [16]:
def binned_mean(timestamps, dict_number, dict_keys, bins, timelabel):
    running_number = {}
    # running_mass = {}
    for i, key in enumerate(dict_keys):
        new_key = 'Exp' + str(i + 1)
        df_number = dict_number[key]
        mean_number, std, errors = bin_mean(timestamps[0][i], df_number, bins, timelabel, None)
        increase_number, std, errors = bin_mean(timestamps[1][i], df_number, bins, timelabel, None)
        bg_number = pd.DataFrame({'Background': mean_number, 'Increase': increase_number}).T
        bg_number.columns = bins
        exp_number = running_mean(df_number, None, bins, timelabel, '10T', 10, timestamps[2][i])
        running_number[new_key] = pd.concat([bg_number, exp_number])

        # if mass == True:
        #     df_mass = pd.DataFrame({'Time': dict_mass[key]['Time']})
        #     mean_mass, std, errors = bin_mean(timestamps[0][i], df_mass, bins, timelabel, None)
        #     increase_mass, std, errors = bin_mean(timestamps[1][i], df_mass, bins, timelabel, None)
        #     bg_mass = pd.DataFrame({'Background': mean_mass, 'Increase': increase_mass}).T
        #     bg_mass.columns = bins
        #     exp_mass = running_mean(df_mass, None, bins, 'Time', '10T', 10, timestamps[2][i])
        #     running_mass[new_key] = pd.concat([bg_mass, exp_mass])

    return running_number   # , running_mass 

In [17]:
binned_0049 = binned_mean(timestamps, LCS_KU, KU_dict_keys[0], LCS_conc, 'timestamp')
binned_0051 = binned_mean(timestamps, LCS_KU, KU_dict_keys[1], LCS_conc, 'timestamp')
binned_0109 = binned_mean(timestamps, LCS_OPC5000, OPC5000_dict_keys, LCS_conc, 'timestamp')
binned_0072 = binned_mean(timestamps, LCS_LUND, Lund_dict_keys[0], Lund_conc, 'Timestamp')
binned_0077 = binned_mean(timestamps, LCS_LUND, Lund_dict_keys[1], Lund_conc, 'Timestamp')

c:\Users\B363580\Documents\GitHub\Master-thesis\calculations.py:277: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  time = pd.to_datetime(df[timelabel])
c:\Users\B363580\Documents\GitHub\Master-thesis\calculations.py:277: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  time = pd.to_datetime(df[timelabel])
c:\Users\B363580\Documents\GitHub\Master-thesis\calculations.py:245: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  time = pd.to_datetime(df[timelabel])
c:\Users\B363580\Documents\GitHub\Master-thesis\calculations.py:277: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was spec

In [19]:
for i in range(4):
    key = 'Exp' + str(i + 1)
    print('Experiment ', i + 1)
    binned_0049[key]['2.5/10'] = binned_0049[key]['PM5000S_2_PN2.5'] / binned_0049[key]['PM5000S_2_PN10']
    binned_0049[key]['1/10'] = binned_0049[key]['PM5000S_2_PN1'] / binned_0049[key]['PM5000S_2_PN10']
    display(binned_0049[key])

Experiment  1


,PM5000S_2_PN0.3,PM5000S_2_PN0.5,PM5000S_2_PN1,PM5000S_2_PN2.5,PM5000S_2_PN5,PM5000S_2_PN10,2.5/10,1/10
Background,158.239745,29.264218,5.578759,0.282077,0.047553,0.000000,inf,inf
Increase,9212.824389,3603.229568,1148.550365,286.868147,97.433221,39.055936,7.345059,29.407831
2024-10-16 10:40:00,27523.854686,11550.877930,3760.247998,884.716455,204.847702,48.822151,18.121210,77.019302
2024-10-16 10:50:00,31193.385547,13056.755273,4168.277783,937.726849,221.544955,54.120105,17.326774,77.019026
2024-10-16 11:00:00,33761.081684,14077.137250,4367.521452,928.223909,209.937440,50.728536,18.297865,86.095949
2024-10-16 11:10:00,35893.496419,14921.742635,4517.067663,910.244351,199.739616,47.589277,19.127089,94.917761
2024-10-16 11:20:00,37728.394948,15668.962584,4661.601699,901.769602,192.223355,45.617401,19.768106,102.189112
2024-10-16 11:30:00,39102.969097,16224.864426,4760.449577,890.758692,185.056798,43.679283,20.393162,108.986441
2024-10-16 11:40:00,40213.196057,16674.793950,4838.569224,882.311983,179.373997,42.091051,20.961985,114.954822
2024-10-16 11:50:00,41130.920671,17080.345219,4920.371283,879.548586,175.228413,40.707762,21.606410,120.870592


Experiment  2


,PM5000S_2_PN0.3,PM5000S_2_PN0.5,PM5000S_2_PN1,PM5000S_2_PN2.5,PM5000S_2_PN5,PM5000S_2_PN10,2.5/10,1/10
Background,293.372655,29.693461,9.089024,2.769335,1.137994,0.459886,6.021781,19.763630
Increase,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-16 15:00:00,140273.881971,62082.376579,18141.860273,2652.121025,460.973447,85.044444,31.185118,213.322110
2024-10-16 15:10:00,154864.328496,68580.396103,19703.716563,2829.251711,483.105368,90.363565,31.309651,218.049350
2024-10-16 15:20:00,161757.711190,72003.259901,20571.950690,2949.270355,507.455315,97.410649,30.276673,211.187902
2024-10-16 15:30:00,166118.167180,74672.032328,21411.981328,3095.326942,538.010195,104.482963,29.625183,204.932753


Experiment  3


,PM5000S_2_PN0.3,PM5000S_2_PN0.5,PM5000S_2_PN1,PM5000S_2_PN2.5,PM5000S_2_PN5,PM5000S_2_PN10,2.5/10,1/10
Background,188.210173,38.179621,5.286316,0.009347,0.000000,0.000000,inf,inf
Increase,5845.900455,2522.203756,745.492675,137.721078,54.490426,33.920869,4.060069,21.977405
2024-10-17 10:20:00,21731.484765,10715.039941,3093.507324,421.043768,71.454474,26.726580,15.753746,115.746471
2024-10-17 10:30:00,24538.199122,12254.564892,3495.295923,463.394012,67.751104,21.180794,21.878029,165.021952
2024-10-17 10:40:00,26513.169076,13338.577929,3777.256869,485.509212,64.519452,17.051310,28.473426,221.522972
2024-10-17 10:50:00,27881.071339,14061.485669,3958.130811,502.190204,63.604952,14.465487,34.716440,273.625828
2024-10-17 11:00:00,28686.142735,14470.667461,4039.543106,502.710422,62.548020,13.001645,38.665140,310.694769
2024-10-17 11:10:00,28966.860938,14600.985270,4049.284355,496.223117,61.018686,11.703208,42.400605,345.997801
2024-10-17 11:20:00,28705.942020,14447.797586,3983.389296,480.244351,57.680594,10.258637,46.813663,388.296172
2024-10-17 11:30:00,28181.484376,14152.166687,3878.411685,462.201369,54.601179,9.366846,49.344395,414.057361


Experiment  4


,PM5000S_2_PN0.3,PM5000S_2_PN0.5,PM5000S_2_PN1,PM5000S_2_PN2.5,PM5000S_2_PN5,PM5000S_2_PN10,2.5/10,1/10
Background,7144.979199,832.497217,140.290926,53.334469,36.139809,25.138244,2.121647,5.580777
Increase,54843.577021,26447.557349,8266.310268,1075.165044,186.633662,39.423957,27.271871,209.677338
2024-10-17 14:30:00,137914.400800,73686.282422,23506.071291,2640.283984,347.375232,53.631335,49.230249,438.289878
2024-10-17 14:40:00,145968.708220,79388.656056,25870.813186,2833.494617,342.889278,50.681547,55.907816,510.458236
2024-10-17 14:50:00,147543.956530,81029.674610,26815.668036,2906.484188,333.087527,47.253094,61.508865,567.490204
2024-10-17 15:00:00,147272.924423,81351.779395,27212.612355,2933.835370,329.329531,44.638894,65.723746,609.616629


In [20]:
for i in range(4):
    key = 'Exp' + str(i + 1)
    print('Experiment ', i + 1)
    binned_0051[key]['2.5/10'] = binned_0051[key]['PM5000S_2_PN2.5'] / binned_0051[key]['PM5000S_2_PN10']
    binned_0051[key]['1/10'] = binned_0051[key]['PM5000S_2_PN1'] / binned_0051[key]['PM5000S_2_PN10']
    display(binned_0051[key])

Experiment  1


,PM5000S_2_PN0.3,PM5000S_2_PN0.5,PM5000S_2_PN1,PM5000S_2_PN2.5,PM5000S_2_PN5,PM5000S_2_PN10,2.5/10,1/10
Background,149.746220,33.708517,5.461729,0.069615,0.000000,0.000000,inf,inf
Increase,11646.333118,4450.556664,1288.149708,304.583750,52.893464,6.743496,45.167038,191.021046
2024-10-16 10:40:00,33801.015624,13837.375976,4200.750244,1002.177368,158.174362,9.918345,101.042801,423.533383
2024-10-16 10:50:00,38008.346874,15553.901612,4607.908887,1063.142890,164.231573,10.101746,105.243478,456.149744
2024-10-16 11:00:00,40806.821614,16719.045801,4797.098796,1060.752094,159.265708,9.607801,110.405291,499.292053
2024-10-16 11:10:00,43260.471192,17738.191382,4957.443774,1054.982478,153.864314,9.325011,113.134721,531.628751
2024-10-16 11:20:00,45248.177969,18589.340996,5100.479160,1053.971775,152.772611,9.118564,115.585278,559.351131
2024-10-16 11:30:00,46659.873438,19194.980811,5185.822673,1050.617630,150.792281,8.841576,118.826959,586.526933
2024-10-16 11:40:00,47682.085603,19638.420243,5253.430636,1047.515554,149.662198,8.605293,121.729207,610.488260
2024-10-16 11:50:00,48534.688477,20049.580823,5323.539075,1051.537795,150.317768,8.418263,124.911495,632.379763


Experiment  2


,PM5000S_2_PN0.3,PM5000S_2_PN0.5,PM5000S_2_PN1,PM5000S_2_PN2.5,PM5000S_2_PN5,PM5000S_2_PN10,2.5/10,1/10
Background,97.200051,32.829776,9.348810,3.243360,0.688235,0.000000,inf,inf
Increase,43691.390325,18428.458399,5188.417149,1060.924002,188.748845,11.927395,88.948508,435.000020
2024-10-16 15:00:00,173008.545320,76766.255079,21652.987012,4260.018286,698.419501,35.402789,120.330020,611.618118
2024-10-16 15:10:00,179643.990635,80030.094727,22053.539308,4274.937512,694.935126,33.736470,126.715616,653.700272
2024-10-16 15:20:00,184165.591673,82463.659767,22480.148211,4320.587899,706.064762,34.165179,126.461738,657.984212
2024-10-16 15:30:00,187525.682043,84645.139845,23055.909131,4450.974371,734.293224,36.191297,122.984659,637.056718


Experiment  3


,PM5000S_2_PN0.3,PM5000S_2_PN0.5,PM5000S_2_PN1,PM5000S_2_PN2.5,PM5000S_2_PN5,PM5000S_2_PN10,2.5/10,1/10
Background,239.322838,46.824340,6.582266,0.060816,0.000000,0.000000,inf,inf
Increase,7858.736515,3388.480896,891.240924,135.885182,20.800478,3.463585,39.232523,257.317458
2024-10-17 10:20:00,26322.033400,13191.703028,3510.741162,538.220267,56.117049,2.602690,206.793849,1348.889521
2024-10-17 10:30:00,29266.124415,14878.312550,3916.992102,590.880672,60.459745,2.332532,253.321589,1679.287734
2024-10-17 10:40:00,31331.163022,16058.708301,4172.771590,624.516181,64.253103,2.066176,302.256974,2019.562263
2024-10-17 10:50:00,32723.447463,16824.765112,4357.848468,652.372305,67.948035,1.921055,339.590690,2268.466574
2024-10-17 11:00:00,33560.537658,17324.315606,4471.842153,678.027849,72.861762,1.743510,388.886598,2564.849636
2024-10-17 11:10:00,33951.862112,17496.138005,4489.329561,677.839878,74.220913,1.527925,443.634152,2938.186401
2024-10-17 11:20:00,34099.079354,17556.157492,4481.018182,674.469895,73.582211,1.450127,465.111071,3090.087760
2024-10-17 11:30:00,33776.202736,17404.162831,4410.447153,655.259074,71.357351,1.385527,472.931168,3183.226306


Experiment  4


,PM5000S_2_PN0.3,PM5000S_2_PN0.5,PM5000S_2_PN1,PM5000S_2_PN2.5,PM5000S_2_PN5,PM5000S_2_PN10,2.5/10,1/10
Background,12195.869042,939.976770,130.757499,29.732450,9.166178,1.514546,19.631264,86.334460
Increase,65684.921424,30391.921603,9030.558226,1385.279398,201.693456,12.170716,113.820695,741.990690
2024-10-17 14:30:00,157950.118770,83478.587502,25202.333399,3586.183203,424.369293,18.301515,195.950077,1377.062714
2024-10-17 14:40:00,165714.828140,89406.896877,27466.201368,3827.121423,441.505458,17.477650,218.972308,1571.504229
2024-10-17 14:50:00,166257.576060,90794.286721,28346.727213,3933.060498,442.311967,16.387625,240.001859,1729.764197
2024-10-17 15:00:00,165048.564870,90749.996290,28715.190918,3987.277185,433.883977,15.958901,249.846606,1799.321356


In [21]:
for i in range(4):
    key = 'Exp' + str(i + 1)
    print('Experiment ', i + 1)
    binned_0109[key]['2.5/10'] = binned_0109[key]['PM5000S_2_PN2.5'] / binned_0109[key]['PM5000S_2_PN10']
    binned_0109[key]['1/10'] = binned_0109[key]['PM5000S_2_PN1'] / binned_0109[key]['PM5000S_2_PN10']
    display(binned_0109[key])

Experiment  1


,PM5000S_2_PN0.3,PM5000S_2_PN0.5,PM5000S_2_PN1,PM5000S_2_PN2.5,PM5000S_2_PN5,PM5000S_2_PN10,2.5/10,1/10
Background,8312.297718,96.582888,5.814617,1.317291,0.144385,0.033868,38.894736,171.684205
Increase,14718.570241,3249.921279,882.246736,216.213861,45.480705,3.377569,64.014646,261.207639
2024-10-16 10:40:00,29053.015626,9916.882618,2879.345068,743.317639,147.368622,7.341176,101.253205,392.218482
2024-10-16 10:50:00,30334.425999,11069.394380,3210.881871,798.943654,156.810601,7.553512,105.771148,425.084622
2024-10-16 11:00:00,31020.040003,11646.556395,3350.812306,814.344808,160.063238,7.083195,114.968575,473.065111
2024-10-16 11:10:00,31775.638303,12218.200915,3466.858308,823.250152,158.273644,6.800573,121.056007,509.789183
2024-10-16 11:20:00,32600.367205,12666.583447,3575.019977,837.415808,158.851465,6.533399,128.174597,547.191419
2024-10-16 11:30:00,34357.660431,13361.415210,3669.943368,846.048787,160.179796,6.278055,134.762885,584.566945
2024-10-16 11:40:00,34697.606653,13586.637465,3726.355801,854.268447,160.713919,6.175407,138.333948,603.418645
2024-10-16 11:50:00,34726.303088,13674.930670,3773.036964,868.235161,166.211594,6.194216,140.168691,609.122594


Experiment  2


,PM5000S_2_PN0.3,PM5000S_2_PN0.5,PM5000S_2_PN1,PM5000S_2_PN2.5,PM5000S_2_PN5,PM5000S_2_PN10,2.5/10,1/10
Background,8704.869914,1743.739817,8.581556,2.276193,0.444350,0.000000,inf,inf
Increase,51954.962477,21020.555421,4673.941770,977.833300,200.391320,8.365553,116.888063,558.712818
2024-10-16 15:00:00,146719.113300,66515.953516,15699.471876,3205.269043,600.619888,21.606283,148.348934,726.616044
2024-10-16 15:10:00,150681.257435,68841.940040,15991.328565,3244.855884,608.313417,21.630220,150.014929,739.304948
2024-10-16 15:20:00,155083.359130,70889.998569,16183.986751,3258.944539,612.159453,21.387637,152.375159,756.698226
2024-10-16 15:30:00,158566.581073,72814.537208,16551.815259,3340.685309,628.852082,22.134760,150.924850,747.774782


Experiment  3


,PM5000S_2_PN0.3,PM5000S_2_PN0.5,PM5000S_2_PN1,PM5000S_2_PN2.5,PM5000S_2_PN5,PM5000S_2_PN10,2.5/10,1/10
Background,9124.613281,342.993989,3.836742,0.327572,0.030769,0.000000,inf,inf
Increase,15414.369697,2732.395329,567.053374,89.354454,12.263922,1.510924,59.138932,375.302285
2024-10-17 10:20:00,28596.162890,10336.837256,2295.199988,361.580396,40.462745,1.698039,212.939957,1351.676672
2024-10-17 10:30:00,32835.826953,11938.145824,2592.547772,400.237622,44.863134,1.524307,262.570289,1700.804671
2024-10-17 10:40:00,34207.036458,12984.335595,2807.077893,431.989851,49.038665,1.322858,326.557993,2121.979768
2024-10-17 10:50:00,35152.225684,13687.781677,2920.532333,453.864821,52.364084,1.257732,360.859801,2322.063023
2024-10-17 11:00:00,35099.431502,14062.801145,2978.913562,462.260784,54.476592,1.140472,405.324249,2612.001590
2024-10-17 11:10:00,34680.014305,14087.761452,2984.895319,462.148843,55.981683,1.093550,422.613423,2729.546657
2024-10-17 11:20:00,33617.754310,13781.598739,2947.639915,454.265591,56.415980,1.048522,433.243856,2811.234018
2024-10-17 11:30:00,32965.748753,13465.010740,2875.695807,443.079103,55.383971,0.992664,446.353483,2896.947368


Experiment  4


,PM5000S_2_PN0.3,PM5000S_2_PN0.5,PM5000S_2_PN1,PM5000S_2_PN2.5,PM5000S_2_PN5,PM5000S_2_PN10,2.5/10,1/10
Background,16977.593129,1811.239674,70.650981,22.830383,6.801627,1.099871,20.757332,64.235710
Increase,68697.777184,30389.864390,7372.565354,1160.620919,185.980392,7.645425,151.805943,964.310758
2024-10-17 14:30:00,135749.158610,71720.995314,17948.742775,2662.400757,362.557138,11.619106,229.139907,1544.761146
2024-10-17 14:40:00,141815.219150,76911.667970,19695.540235,2876.218701,382.678726,10.829886,265.581619,1818.628558
2024-10-17 14:50:00,140887.044017,77449.030990,20300.319076,2966.613517,387.918905,10.129047,292.881810,2004.168778
2024-10-17 15:00:00,139526.486150,76976.554493,20555.440480,3033.281213,392.588688,9.807569,309.279610,2095.875117


In [22]:
for i in range(4):
    key = 'Exp' + str(i + 1)
    print('Experiment ', i + 1)
    binned_0072[key]['2.5/10'] = binned_0072[key]['PN2.5, #/cm3'] / binned_0072[key]['PN10, #/cm3']
    binned_0072[key]['1/10'] = binned_0072[key]['PN1, #/cm3'] / binned_0072[key]['PN10, #/cm3']
    display(binned_0072[key])

Experiment  1


,"PN0.5, #/m3","PN1, #/cm3","PN2.5, #/cm3","PN4, #/cm3","PN10, #/cm3",2.5/10,1/10
Background,0.458429,0.530456,0.532346,0.532498,0.532611,0.999502,0.995954
Increase,4.197063,5.269920,5.532224,5.585569,5.599013,0.988071,0.941223
2024-10-16 10:40:00,9.741946,14.818893,16.960641,17.409546,17.518376,0.968163,0.845906
2024-10-16 10:50:00,10.135672,16.544683,19.440791,20.049460,20.196505,0.962582,0.819185
2024-10-16 11:00:00,10.436323,17.950944,21.475495,22.217276,22.396156,0.958892,0.801519
2024-10-16 11:10:00,10.943031,19.021603,22.835267,23.638085,23.831625,0.958192,0.798166
2024-10-16 11:20:00,11.390985,19.924712,23.968243,24.819565,25.024763,0.957781,0.796200
2024-10-16 11:30:00,11.895208,20.687661,24.839646,25.713694,25.924404,0.958157,0.797999
2024-10-16 11:40:00,12.311300,21.342318,25.598653,26.494604,26.710613,0.958370,0.799020
2024-10-16 11:50:00,12.609486,21.900412,26.284238,27.207064,27.429541,0.958246,0.798424


Experiment  2


,"PN0.5, #/m3","PN1, #/cm3","PN2.5, #/cm3","PN4, #/cm3","PN10, #/cm3",2.5/10,1/10
Background,0.230307,0.388491,0.461739,0.477147,0.480865,0.960225,0.807900
Increase,12.109607,21.697364,26.301571,27.271413,27.505026,0.956246,0.788851
2024-10-16 15:00:00,44.110258,81.820512,100.242793,104.125641,105.060221,0.954146,0.778796
2024-10-16 15:10:00,48.290013,88.673116,108.307398,112.445006,113.441126,0.954745,0.781666
2024-10-16 15:20:00,50.547249,93.440268,114.361086,118.770310,119.831661,0.954348,0.779763
2024-10-16 15:30:00,52.113744,96.244810,117.759912,122.294313,123.385814,0.954404,0.780031


Experiment  3


,"PN0.5, #/m3","PN1, #/cm3","PN2.5, #/cm3","PN4, #/cm3","PN10, #/cm3",2.5/10,1/10
Background,0.250250,0.290404,0.291931,0.292119,0.292206,0.999060,0.993834
Increase,3.231120,4.017990,4.196771,4.232929,4.242106,0.989313,0.947169
2024-10-17 10:20:00,7.839239,12.231982,14.137581,14.537443,14.634241,0.966062,0.835847
2024-10-17 10:30:00,7.552628,13.536214,16.410069,17.015424,17.161244,0.956228,0.788766
2024-10-17 10:40:00,7.716564,14.685795,18.129166,18.855210,19.029875,0.952669,0.771723
2024-10-17 10:50:00,7.864168,15.389101,19.148651,19.941667,20.132349,0.951138,0.764397
2024-10-17 11:00:00,8.038629,15.943171,19.912155,20.749492,20.950786,0.950425,0.760982
2024-10-17 11:10:00,8.076725,16.113283,20.157112,21.010299,21.215386,0.950118,0.759509
2024-10-17 11:20:00,7.936289,15.960287,20.009170,20.863505,21.068843,0.949704,0.757530
2024-10-17 11:30:00,7.744655,15.646801,19.640526,20.483269,20.685805,0.949469,0.756403


Experiment  4


,"PN0.5, #/m3","PN1, #/cm3","PN2.5, #/cm3","PN4, #/cm3","PN10, #/cm3",2.5/10,1/10
Background,5.883182,6.807535,6.831791,6.833738,6.835191,0.999503,0.995954
Increase,21.916523,38.474549,46.336732,47.992161,48.391139,0.957546,0.795074
2024-10-17 14:30:00,40.293908,86.550055,110.376318,115.407262,116.615366,0.946499,0.742184
2024-10-17 14:40:00,42.579453,93.319014,119.598800,125.148830,126.481251,0.945585,0.737809
2024-10-17 14:50:00,43.575565,96.039531,123.252581,128.999988,130.379707,0.945336,0.736614
2024-10-17 15:00:00,42.994454,95.402180,122.633625,128.385245,129.765869,0.945038,0.735187


In [23]:
for i in range(4):
    key = 'Exp' + str(i + 1)
    print('Experiment ', i + 1)
    binned_0077[key]['2.5/10'] = binned_0077[key]['PN2.5, #/cm3'] / binned_0077[key]['PN10, #/cm3']
    binned_0077[key]['1/10'] = binned_0077[key]['PN1, #/cm3'] / binned_0077[key]['PN10, #/cm3']
    display(binned_0077[key])

Experiment  1


,"PN0.5, #/m3","PN1, #/cm3","PN2.5, #/cm3","PN4, #/cm3","PN10, #/cm3",2.5/10,1/10
Background,0.278768,0.490331,0.589963,0.611166,0.616473,0.956998,0.795382
Increase,3.343424,5.147523,5.914656,6.077247,6.118104,0.966747,0.841359
2024-10-16 10:40:00,9.050890,14.235491,16.487649,16.965411,17.085365,0.965016,0.833198
2024-10-16 10:50:00,10.284596,16.396324,19.084065,19.654516,19.797670,0.963955,0.828195
2024-10-16 11:00:00,11.325978,18.114416,21.108064,21.743513,21.902959,0.963708,0.827031
2024-10-16 11:10:00,12.177890,19.459057,22.667447,23.348456,23.519340,0.963779,0.827364
2024-10-16 11:20:00,12.758087,20.491433,23.914101,24.640722,24.823019,0.963384,0.825501
2024-10-16 11:30:00,13.125201,21.139430,24.694631,25.449459,25.638815,0.963174,0.824509
2024-10-16 11:40:00,13.558785,21.862964,25.550312,26.333227,26.529622,0.963086,0.824096
2024-10-16 11:50:00,13.982526,22.463946,26.218528,27.015620,27.215595,0.963364,0.825407


Experiment  2


,"PN0.5, #/m3","PN1, #/cm3","PN2.5, #/cm3","PN4, #/cm3","PN10, #/cm3",2.5/10,1/10
Background,0.243842,0.405270,0.478634,0.494225,0.498133,0.960856,0.813578
Increase,11.990202,20.049153,23.727320,24.509143,24.705052,0.960424,0.811541
2024-10-16 15:00:00,51.335396,84.659370,99.718851,102.918591,103.720699,0.961417,0.816224
2024-10-16 15:10:00,54.187378,90.407354,106.912930,110.421079,111.300209,0.960582,0.812284
2024-10-16 15:20:00,56.186879,94.197743,111.577456,115.271884,116.197572,0.960239,0.810669
2024-10-16 15:30:00,57.472996,96.764970,114.782306,118.612708,119.572362,0.959940,0.809259


Experiment  3


,"PN0.5, #/m3","PN1, #/cm3","PN2.5, #/cm3","PN4, #/cm3","PN10, #/cm3",2.5/10,1/10
Background,0.392114,0.452169,0.453736,0.453854,0.453937,0.999558,0.996106
Increase,3.393161,4.050914,4.145025,4.163313,4.168317,0.994412,0.971835
2024-10-17 10:20:00,9.211028,13.430090,15.105282,15.459270,15.548487,0.971495,0.863755
2024-10-17 10:30:00,9.145980,14.669674,17.111542,17.629917,17.759973,0.963489,0.825996
2024-10-17 10:40:00,9.246021,15.705376,18.684569,19.318075,19.476756,0.959327,0.806365
2024-10-17 10:50:00,9.323354,16.411469,19.750882,20.461563,20.639433,0.956949,0.795151
2024-10-17 11:00:00,9.455215,16.935382,20.492263,21.249490,21.438944,0.955843,0.789935
2024-10-17 11:10:00,9.529994,17.225584,20.901761,21.684519,21.880328,0.955276,0.787264
2024-10-17 11:20:00,9.474216,17.078014,20.705401,21.477730,21.670941,0.955445,0.788061
2024-10-17 11:30:00,9.310486,16.752323,20.299196,21.054357,21.243280,0.955558,0.788594


Experiment  4


,"PN0.5, #/m3","PN1, #/cm3","PN2.5, #/cm3","PN4, #/cm3","PN10, #/cm3",2.5/10,1/10
Background,6.376889,7.368945,7.403408,7.407254,7.409071,0.999236,0.994584
Increase,25.769535,43.282838,51.300540,53.004960,53.432002,0.960109,0.810055
2024-10-17 14:30:00,44.213731,87.206831,108.515156,113.058410,114.193410,0.950275,0.763677
2024-10-17 14:40:00,46.390147,93.914986,117.681361,122.750332,124.016271,0.948919,0.757280
2024-10-17 14:50:00,46.535241,95.458988,120.029104,125.270287,126.579049,0.948254,0.754145
2024-10-17 15:00:00,46.252275,95.963068,121.016501,126.361448,127.695966,0.947692,0.751496
